In [1]:
from shared.preface import *
import shared.functions as fct

In [2]:
# c = fct.c_vir(0, Mvir_NFW)
c = fct.c_vir(0, 3.76e12*Msun)
print(c)

19.330215423529488
16.758793969849247


### Critical density of universe.

In [3]:
rho_crit_today = fct.rho_crit(0)
rho_crit_today *= (Msun/kpc**3) / (kg/m**3)
print(f'{rho_crit_today:.2e} kg/m^3') 

8.53e-27 kg/m^3


### Time Variable s(z) and comparison to age of universe.

In [6]:
# In s_of_z function we use
H0_mod = H0/ (1/s)
print(H0_mod)

test_z = 4
s_val = fct.s_of_z(test_z)
print(f'Value of time variable s in seconds at redhshift {test_z}:','\n', s_val)
print('Age of universe in seconds:','\n', t0/s, 1/H0/s)

2.184057031756319e-18
Value of time variable s in seconds at redhshift 4: 
 1.8061156839257661e+18
Age of universe in seconds: 
 4.3519680000000006e+17 4.578635014836794e+17


### 